In [ ]:
import numpy as np
import random

In [3]:
def read_dataset_x(fname):
    f=open(fname, encoding="utf-8")
    dataset=[]
    vpointer=0
    vocabulary={}
    for line in f:
        line=line.replace("{","")
        line=line.replace("}","")
        line=line.split(":")[1]
        line=line.replace("\"","")
        words=line.split()
        words.insert(0,"BOS")
        words.insert(len(words),"EOS")
        for word in words:
            if word not in vocabulary:
                vocabulary[word]=vpointer
                vpointer+=1
        words=[vocabulary[word] for word in words]
        dataset.append(words)
    return dataset,vocabulary
def read_dataset_y(fname):
    f=open(fname, encoding="utf-8")
    dataset=[]
    vpointer=0
    vocabulary={}
    for line in f:
        line=line.replace("{","")
        line=line.replace("}","")
        line=line.split(":")[1]
        line=line.replace("\"","")
        words=line.split()
        for word in words:
            if word not in vocabulary:
                vocabulary[word]=vpointer
                vpointer+=1
        words=[vocabulary[word] for word in words]
        dataset.append(words)
    return dataset,vocabulary


{')': 17,
 '.': 13,
 'BOS': 0,
 'Bri\\u00fain': 24,
 'Deshalb': 19,
 'EOS': 4,
 'Fiachrach': 22,
 'Florida': 12,
 'Lacus': 1,
 'N\\u00e9ill': 25,
 'S\\u00edl': 26,
 'Siehe': 14,
 'U\\u00ed': 21,
 '\\u00b7': 23,
 '\\u03a0\\u03bb\\u03b1\\u03c4\\u03b5\\u03af\\u03b1': 6,
 '\\u03c3\\u03c4\\u03b7\\u03bd': 5,
 '\\u03c4\\u03bf\\u03c5': 7,
 '\\u0421\\u0438\\u0446\\u0437\\u044f\\u043d': 16,
 '\\u0440\\u0430\\u0441\\u043f\\u043e\\u043b\\u043e\\u0436\\u0435\\u043d': 18,
 '\\u05dc\\u05d7\\u05e9\\u05d5\\u05d1': 10,
 '\\u05de\\u05e4\\u05e0\\u05d9': 8,
 '\\u05e9\\u05d8\\u05e2\\u05d5': 9,
 '\\u2022': 11,
 'auch': 15,
 'class': 2,
 'fames': 3,
 'wird': 20}

In [ ]:
class TextIterator:
    def __init__(self,x_dataset,y_dataset,batch_size=20):
        self.x=x_dataset
        self.y=y_dataset
        self.n=len(x_dataset)
        self.indices=[i for i in range(self.n)]
        random.shuffle(self.indices)
        self.current_index=0
        self.batch_size=batch_size
    def __iter__(self):
        return self
    def reset(self):
        self.current_index=0
    def get_elements_from_index(self,indices):
        lx=[]
        ly=[]
        for i in indices:
            lx.append(self.x[i]) 
            ly.append(self.y[i])
        return lx,ly
    def __next__(self):
        if self.current_index==-1:
            self.reset()
            raise StopIteration
        if (self.current_index+self.batch_size)<self.n:
            indices=self.indices[self.current_index:self.current_index+self.batch_size]
            x,y=self.get_elements_from_index(indices)
            self.current_index+=self.batch_size
            if self.current_index==self.n:
                self.reset()
                raise StopIteration
        else:
            size=self.n-self.current_index
            indices=self.indices[self.current_index:self.current_index+size]
            x,y=self.get_elements_from_index(indices)
            self.current_index=-1
        return x,y

In [ ]:
x,_=read_dataset('data/sample.txt')
y,_=read_dataset('data/sampley.txt')
t=TextIterator(x,y,4)
for log,lab in t:
    print(log,lab)

In [ ]:
def read_test_dataset(fname,vocabulary):
    f=open(fname, encoding="utf-8")
    dataset=[]
    cnt1=0
    cnt2=0
    for line in f:
        sent=[]
        line=line.replace("{","")
        line=line.replace("}","")
        line=line.split(":")[1]
        line=line.replace("\"","")
        words=line.split()
        for word in words:
            if word in v.keys():
                sent.append(vocabulary[word])
            else:
                sent.append(vocabulary["UNK"])
        dataset.append(sent)
    return dataset
#read_test_dataset("data/test_X_languages_homework.json.txt",v)

In [ ]:
def add_extras(vocabulary):
    n=len(vocabulary)
    vocabulary["UNK"]=n
    return vocabulary

In [ ]:
v=add_unknown(v)

In [4]:
def prepare_input_feed(model,x,y,max_sen_len):
    n_sen=len(x)
    for i in range(n_sen):
        if len(x[i])>max_sen_len:
            x.pop(i)
            y.pop(i)
    n_sen=len(x)
    seq_len_x=[len(xi) for xi in x]
    max_len_x=max(seq_len_x)
    x_arr=np.zeros(n_sen,max_len_x)
    x_mask=np.zeros(n_sen,max_len_x)
    for i,xi in enumerate(x):
        for j,xj in enumerate(xi):
            x_arr[i][j]=xj
            x_mask[i][j]=1
    return {
        model.x_inputs: x_arr,
        model.x_mask: x_mask,
        model.y: y,
    }